In [ ]:
# Script to gather VASP input files and modify parameters for KPOINTS and INCAR
# Requires locally saved POSCAR file, and reference KPOINTS and INCAR files
# POTCAR file is generated by appending POTCAR files (on server) for each element
# Server side .sbatch file is utilized

In [1]:
import numpy as np
import pandas as pd

class Job:
    '''
    Individual job to be submitted on slurm capable remote server. Methods are provided for loading locally saved reference INCAR and KPOINTS files and modifying parameters.

    Parameters
    ----------
    name:    str,
        Name of job which also is going to be the remote folder name.
    poscar_path:   str
        Path to local POSCAR file.
    '''
    def __init__(self, name: str, poscar_path_local: str):
        self.name = name
        self.poscar_path = poscar_path_local
        print(f'{self.name} Job initialized')
        
    def elements_poscar(self):
        '''Extract list of elements from local POSCAR file'''
        self.elements = np.loadtxt(self.poscar_path,dtype=str,skiprows=5, max_rows=1)

    def incar_load(self, incar_path_reference: str):
        '''Load reference INCAR file as pandas data frame'''
        self.incar = pd.read_csv(incar_path_reference, names=['Setting','Value'], sep=' = ', dtype=str, engine='python')
    
    def incar_modify(self, setting: str, value: str):
        '''Modify individual values of a setting after incar is loaded into instance'''
        self.incar.loc[self.incar['Setting'] == setting,'Value'] = value

    def kpoints_load(self, kpoints_path_reference: str):
        '''Load reference KPOINTS file as pandas data frame'''
        self.kpoints = pd.read_csv(kpoints_path_reference, names=['Value','Description'], sep=' ! ', dtype=str, engine='python')
    
    def kpoints_modify(self, description: str, value: str):
        '''Modify individual values after kpoints is loaded into instance'''
        self.kpoints.loc[self.kpoints['Description'] == description,'Value'] = value


In [2]:
import paramiko as pm
import socket as socket
import os

class Server:
    '''Server class offers methods for connecting to the server, creating folders, and uploading files'''
    def __init__(self, username: str, host: str, home_path_remote: str, batch_path_remote: str):
        self.host = host
        self.user = username
        self.home_path = home_path_remote # SFTP home path
        self.batch_path = batch_path_remote # e.g. ~/batch_scripts/vasp.sbatch

    def connect(self):
        '''Connects to server and returns SFTP and transport objects'''
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((self.host, 22))
        ts = pm.Transport(sock)
        ts.start_client(timeout=10)
        ts.auth_interactive_dumb(self.user)
        print(ts)
        sftp = pm.SFTPClient.from_transport(ts)
        print(sftp)
        return sftp, ts
        
    def create_folder(self, sftp: object, name: str):
        '''Construct folder in home directory'''
        sftp.mkdir(name)

    def potcar_gather(self, job: Job, sftp:object, potcar_folder_path: str, suffix_to_element:str=''):
        '''
        Gathers and appends POTCAR files for the elements. Handling is server-side.

        Parameters
        ----------
        job:    Job
            Job instance
        sftp: object
            Paramiko SFTP object
        potcar_folder_path:   str
            Remote path to POTCAR folder. Requires potential files contained there to be named after their respective Element (e.g. Li).
        suffix_to_element:  str=''
            Added suffix to name of potentials. E.g. to choose X_GW potentials, assign "_GW".
        '''
        file = sftp.open(f'{potcar_folder_path}/{job.elements[0]}/POTCAR', 'r') # open POTCAR for first element in reading mode
        sftp.putfo(file, f'{self.home_path}/{job.name}/POTCAR', confirm=True) # save as POTCAR in job folder
        print(f'Created POTCAR for {job.elements[0]}')
        file.close() # close initial INCAR file
        file = sftp.file(f'{self.home_path}/{job.name}/POTCAR', 'a') # open new POTCAR file in appending mode

        for element in job.elements[1:]:
            path = f'{potcar_folder_path}/{element}{suffix_to_element}/POTCAR'
            sftp.getfo(path, file)
            print(f'Appended POTCAR for {element}')
            
        file.close() # close final INCAR file
        print('POTCAR files successfully merged and copied to job folder')
        
    def incar_upload(self, job: Job, sftp:object, tempfolder_path_local: str):
        '''Upload local INCAR file, must be loaded by Job.incar_load first'''
        np.savetxt(f'{tempfolder_path_local}/INCAR',job.incar, fmt='%1s', delimiter=' = ')
        print('INCAR file temporarily saved')
        
        try:
            sftp.put(f'{tempfolder_path_local}/INCAR', f'{self.home_path}/{job.name}/INCAR', confirm=True)
            print('INCAR file uploaded')
        except Exception as e:
            print(f'Failed to upload INCAR file. {e}')

        os.remove(f'{tempfolder_path_local}/INCAR')
        if not os.path.exists(f'{tempfolder_path_local}/INCAR'): print('Temporarirly saved INCAR removed')

    def poscar_upload(self, job: Job, sftp: object):
        '''Upload local POSCAR file'''
        try:
            sftp.put(job.poscar_path, f'{self.home_path}/{job.name}/POSCAR', confirm=True)
            print('POSCAR file uploaded')
        except Exception as e:
            print(f'Failed to upload POSCAR file. {e}')
        
    def kpoints_upload(self, job: Job, sftp: object, tempfolder_path_local: str):
        '''Upload local KPOINTS file, must be loaded by Job.kpoints_load first'''
        np.savetxt(f'{tempfolder_path_local}/KPOINTS', job.kpoints['Value'], fmt='%1s', delimiter=' ! ')
        print('KPOINTS file temporarily saved')

        try:
            sftp.put(tempfolder_path_local+'/KPOINTS', f'{self.home_path}/{job.name}/KPOINTS', confirm=True)
            print('KPOINTS file uploaded')
        except Exception as e:
            print(f'Failed to upload KPOINTS file. {e}')

        os.remove(f'{tempfolder_path_local}/KPOINTS')
        if not os.path.exists(tempfolder_path_local+'/KPOINTS'): print('Temporarirly saved KPOINTS removed')

    def name_batch(self, job:Job, ts: object):
        '''Give name for job to .sbatch file - needs #SBATCH -J line which specifies name beforehand'''
        ch= ts.open_session(timeout=15)
        ch.exec_command(f"sed -i '/SBATCH -J/c\#SBATCH -J {job.name}' {self.batch_path}")
        ret = ch.recv_exit_status()
        ch.close()
        print(f'Updating name - Exit status of execution command: {ret}')

    def batch_submit(self, job: Job, ts: object):
        '''Submit batch for calculation'''
        ch= ts.open_session(timeout=15)
        ch.exec_command(f'cd ~/{job.name} \n sbatch {self.batch_path}')
        ret = ch.recv_exit_status()
        ch.close()
        print(f'Submitting batch - Exit status of execution command: {ret}')

    def exec_command(self, ts: object, command: str):
        '''!!NOT YET TESTED!! Execution of a general SSH command while obtaining list of returns'''
        key = True
        answers=[]
        chan= ts.open_session(timeout=15)

        chan.exec_command(command)
        while key:
            if chan.recv_ready():
                answer = chan.recv(4096).decode('ascii')
                answers.append(answer)
                print("recv:\n%s" % answer)
            if chan.recv_stderr_ready():
                answer = chan.recv_stderr(4096).decode('ascii')
                answers.append(answer)
                print("error:\n%s" % answer)
            if chan.exit_status_ready():
                print("exit status: %s" % chan.recv_exit_status())
                key = False

        return answers
        

/Users/dominicwelti/opt/anaconda3/lib/python3.9/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Initialize Job
job = Job('MD_ML_Test_MRB2_12_550eV_3x5x6_bulk_4c',poscar_path_local='/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/Ex_4b_poscar_beta_LPS')

job.elements_poscar()
job.incar_load('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/incar_MD_ML')
job.kpoints_load('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/kpoints_reference')

MD_ML_Test_MRB2_12_550eV_3x5x6_bulk_4c Job instance started


In [3]:
job.elements

array(['S', 'Li', 'P'], dtype='<U2')

In [51]:
# Modify INCAR and KPOINTS
job.incar_modify('ENCUT','550')
job.incar_modify('ISMEAR','0') # Gaussian smearing
job.incar_modify('SIGMA','0.2')
job.incar_modify('PMASS','1000')
job.kpoints_modify('subdivisions along recipr. l.', '3 5 6')

In [3]:
# Initialize Server
# batch-file for shared memory (ipcs & system-V): /home/st/st_us-031400/st_st179390/batch_scripts/vasp.6.4.1_sysv.sbatch
# batch-file for MPI-3 shared-memory capabilities: /home/st/st_us-031400/st_st179390/batch_scripts/vasp.6.4.1_shmem.sbatch
# ipcs shared memory and MPI 2021.9.0. : /home/st/st_us-031400/st_st179390/batch_scripts/vasp.6.4.1_sysv_mpi2021.9.0.sbatch
server = Server('st_st179390','justus2.uni-ulm.de', '/lustre/home/st/st_us-031400/st_st179390', '/home/st/st_us-031400/st_st179390/batch_scripts/vasp.6.4.1.sbatch')
sftp, ts = server.connect()

Your OTP: Password: <paramiko.Transport at 0x26887430 (cipher aes128-ctr, 128 bits) (active; 0 open channel(s))>


In [4]:
# Testing commands and getting return strings

key = True
chan= ts.open_session(timeout=15)

chan.exec_command('ls')
while key:
    if chan.recv_ready():
        print("recv:\n%s" % chan.recv(4096).decode('ascii'))
    if chan.recv_stderr_ready():
        print("error:\n%s" % chan.recv_stderr(4096).decode('ascii'))
    if chan.exit_status_ready():
        print("exit status: %s" % chan.recv_exit_status())
        key = False

recv:
batch_scripts
bulk_LPS_12_x_10_x_6_280eV_Ex_4b
bulk_LPS_12_x_10_x_6_280eV_Ex_4c
bulk_LPS_6x10x12_100eV
bulk_LPS_6x10x12_140eV
bulk_LPS_6x10x12_180eV
bulk_LPS_6x10x12_220eV
bulk_LPS_6x10x12_260eV
bulk_LPS_6_x_10_x_12_280eV
bulk_LPS_6_x_10_x_12_280eV_GW
bulk_LPS_6_x_10_x_12_280eV_new_structure
bulk_LPS_6x10x12_300eV
bulk_LPS_6x10x12_340eV
bulk_LPS_6x10x12_380eV
bulk_LPS_6x10x12_420eV
bulk_LPS_6x10x12_460eV
bulk_LPS_6x10x12_500eV
bulk_LPS_6x10x12_540eV
bulk_LPS_6x10x12_580eV
bulk_LPS_6x10x12_620eV
bulk_LPS_6x10x12_660eV
bulk_LPS_6x10x12_700eV
bulk_LPS_6x10x12_740eV
bulk_LPS_LREAL_auto_mesh_1x1x1_500eV
bulk_LPS_LREAL_auto_mesh_1x2x2_500eV
bulk_LPS_LREAL_auto_mesh_2x2x2_500eV
bulk_LPS_LREAL_auto_mesh_2x2x4_500eV
bulk_LPS_LREAL_auto_mesh_3x4x6_500eV
bulk_LPS_LREAL_auto_mesh_3x5x6_500eV
bulk_LPS_mesh_1x1x1_500eV
bulk_LPS_mesh_1x2x2_500eV
bulk_LPS_mesh_2x2x2_500eV
bulk_LPS_mesh_2x2x4_500eV
bulk_LPS_mesh_3x4x6_500eV
bulk_LPS_mesh_3x5x6_500eV
bulk_LPS_mesh_4x4x4_500eV
bulk_LPS_mesh_4x5x8_5

In [114]:
#add_line = pd.DataFrame({
#    'Setting': ['LREAL'],
#    'Value': ['Auto']
#})

#job.incar = pd.concat([job.incar, add_line])

In [27]:
job.kpoints

,Value,Description
0,Automatic mesh,descriptive line
1,0,k-points (0: automatic generation)
2,Gamma,Gamma centered grid
3,3 5 6,subdivisions along recipr. l.
4,0. 0. 0.,optional shift of the mesh


In [58]:
job.incar

,Setting,Value
0,ISTART,0
1,ISPIN,1
2,GGA,PE
3,ISMEAR,0
4,SIGMA,0.2
5,ENCUT,550
6,PREC,accurate
7,ADDGRID,.TRUE.
8,EDIFF,1.0E-7
9,EDIFFG,-5.0E-3


In [57]:
job.incar_modify('ML_LMLFF','.TRUE.')

In [59]:
job.name = 'ML_Test_on_3x5x6_550eV_bulk_4b'

In [60]:
server.create_folder(sftp, job.name)
server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
server.poscar_upload(job,sftp)
server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
server.name_batch(job, ts)
server.batch_submit(job, ts)

Created POTCAR for S
Appended POTCAR for Li
Appended POTCAR for P
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0


In [33]:
#upload and start jobs for RCUT1 & 2 evaluation

job.incar_modify('ML_LMLFF','.TRUE.')
parameters = pd.DataFrame({
    'ML_RCUT1': [8.0, 10.5, 13.0],
    'ML_RCUT2': [5.0, 7.5, 10.0]
})

for rcut1 in parameters['ML_RCUT1']:
    for rcut2 in parameters['ML_RCUT2']:
        job.incar_modify('ML_RCUT1',rcut1)
        job.incar_modify('ML_RCUT2',rcut2)

        k_points = job.kpoints.loc[3,"Value"].replace(' ', 'x')
        e_cut = float(job.incar.loc[job.incar['Setting'] == 'ENCUT','Value'].values[0])
        job.name = f'ML_Test_RCUT1_{rcut1}A_RCUT2_{rcut2}A_{k_points}_{int(e_cut)}eV_bulk_4b'
        print(job.name)

        #server.create_folder(sftp, job.name)
        #server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
        #server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
        #server.poscar_upload(job,sftp)
        #server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
        server.name_batch(job, ts)
        server.batch_submit(job, ts)

ML_Test_RCUT1_8.0A_RCUT2_5.0A_3x5x6_550eV_bulk_4b
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
ML_Test_RCUT1_8.0A_RCUT2_7.5A_3x5x6_550eV_bulk_4b
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
ML_Test_RCUT1_8.0A_RCUT2_10.0A_3x5x6_550eV_bulk_4b
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
ML_Test_RCUT1_10.5A_RCUT2_5.0A_3x5x6_550eV_bulk_4b
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
ML_Test_RCUT1_10.5A_RCUT2_7.5A_3x5x6_550eV_bulk_4b
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
ML_Test_RCUT1_10.5A_RCUT2_10.0A_3x5x6_550eV_bulk_4b
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
ML_Test_RCUT1_13.0A_RCUT2_5.0A_3x5x6_550eV_bulk_4b
Up

In [16]:
server.name_batch(job, ts)

Updating name - exit status of execution command: 0


In [17]:
# upload and start MD jobs for PMASS evaluation

parameters = pd.DataFrame({
    'PMASS': ['500', '750', '1000', '1250', '1500'],
})


for pmass in parameters['PMASS']:
    job.incar_modify('PMASS',pmass)
    k_points = job.kpoints.loc[3,"Value"].replace(' ', 'x')
    e_cut = float(job.incar.loc[job.incar['Setting'] == 'ENCUT','Value'].values[0])
    job.name = f'MD_Test_PMASS_{pmass}amu_{k_points}_{int(e_cut)}eV_bulk_4b'
    print(job.name, pmass)

    server.create_folder(sftp, job.name)
    server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
    server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.poscar_upload(job,sftp)
    server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.name_batch(job, ts)
    server.batch_submit(job, ts)

MD_Test_PMASS_500amu_3x5x6_460eV_bulk_4b 500
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
MD_Test_PMASS_750amu_3x5x6_460eV_bulk_4b 750
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
MD_Test_PMASS_1000amu_3x5x6_460eV_bulk_4b 1000
Created POTCAR for 

In [9]:
# upload and start MD jobs for TEBEG evaluation

parameters = pd.DataFrame({
    'TEBEG': ['900', '1100', '1200', '850'],
})


for tebeg in parameters['TEBEG']:
    job.incar_modify('TEBEG',tebeg)
    k_points = job.kpoints.loc[3,"Value"].replace(' ', 'x')
    e_cut = float(job.incar.loc[job.incar['Setting'] == 'ENCUT','Value'].values[0])
    job.name = f'MD_Test_TEBEG_{tebeg}k_{k_points}_{int(e_cut)}eV_bulk_4b'
    print(job.name, tebeg)

    server.create_folder(sftp, job.name)
    server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
    server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.poscar_upload(job,sftp)
    server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.name_batch(job, ts)
    server.batch_submit(job, ts)

MD_Test_TEBEG_900k_3x5x6_460eV_bulk_4b 900
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
MD_Test_TEBEG_1100k_3x5x6_460eV_bulk_4b 1100
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
MD_Test_TEBEG_1200k_3x5x6_460eV_bulk_4b 1200
Created POTCAR for Li
A

In [117]:
# upload and start jobs for mesh evaluation
for mesh in parameters['k-points']:
    job.kpoints_modify('subdivisions along recipr. l.', mesh)
    k_points = job.kpoints.loc[3,"Value"].replace(' ', 'x')
    e_cut = float(job.incar.loc[job.incar['Setting'] == 'ENCUT','Value'].values[0])
    job.name = f'bulk_LPS_LREAL_auto_mesh_{k_points}_{int(e_cut)}eV'
    print(k_points)

    server.create_folder(sftp, job.name)
    server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
    server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.poscar_upload(job,sftp)
    server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.batch_submit(job, ts)

1x1x1
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
Temporarirly saved INCAR removed
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0
2x2x2
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
Temporarirly saved INCAR removed
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0
2x2x4
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
Temporarirly saved INCAR removed
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0
1x2x2
Created POTCAR for Li
Appended POTCAR for P
Ap

In [46]:
# upload and start jobs for energy cutoff evaluation
for ecut in parameters['Energy cutoff [eV]']:
    job.incar_modify('ENCUT', str(ecut))
    k_points = job.kpoints.loc[3,"Value"].replace(' ', 'x')
    job.name = f'bulk_LPS_{k_points}_{int(ecut)}eV'

    server.create_folder(sftp, job.name)
    server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
    server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.poscar_upload(job,sftp)
    server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.batch_submit(job, ts)


Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
Temporarirly saved INCAR removed
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
Temporarirly saved INCAR removed
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
Temporarirly saved INCAR removed
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0


In [37]:
# create and start single job

server.create_folder(sftp, job.name)
server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
server.poscar_upload(job, sftp)
server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
server.batch_submit(job, ts)

Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0


In [61]:
# Terminate connection
if sftp: sftp.close()
if ts.active: ts.close()